# Tutorial 6: The Representable Perspective

**Course 3: Document Functors (Lorren Dray)**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/category-theory-document-functors/blob/main/notebooks/06_representable_perspective.ipynb)

---

## Overview

In Year 935, Dray discovered a special class of functors: **representable functors**. For any access method A, the functor Hom(A, -) captures the view from that single vantage point.

This discovery laid the groundwork for the Yoneda Lemma, which Pelleth Strand would later call the "Probing Lemma."

### Learning Goals

1. Understand Hom-functors and what they represent
2. See how representable functors capture "perspectives"
3. Connect to the idea that probing fully determines identity
4. Preview the Yoneda perspective

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

archive_structure = pd.read_csv(BASE_URL + "archive_category_structure.csv")
embeddings = pd.read_csv(BASE_URL + "embedding_correspondences.csv")
correspondence = pd.read_csv(BASE_URL + "dray_correspondence.csv")

## Part 1: The Hom-Functor

For any object A in a category C, the **Hom-functor** Hom(A, -) is defined as:

- On objects: Hom(A, B) = {set of morphisms from A to B}
- On morphisms: For f: B → C, Hom(A, f) maps g ∈ Hom(A, B) to f ∘ g ∈ Hom(A, C)

In the Archive context:
- Hom(subject_catalog, -) gives all the ways to "flow" from subject_catalog to other access methods
- This is the "subject-centric" view of the archive

In [ ]:
# Extract morphisms from the archive structure
morphisms = archive_structure[archive_structure['object_type'] == 'morphism']

def hom_from(source, morphisms_df):
    """
    Compute Hom(source, -) for the Archive category.
    Returns all morphisms with the given source.
    """
    result = morphisms_df[morphisms_df['morphism_source'] == source]
    return result[['morphism_target', 'morphism_name', 'notes']]

# Compute Hom(subject_catalog, -)
print("Hom(subject_catalog, -) — The Subject-Centric Perspective:\n")
hom_subject = hom_from('subject_catalog', morphisms)

for _, row in hom_subject.iterrows():
    print(f"  Hom(subject_catalog, {row['morphism_target']}) contains:")
    print(f"    {row['morphism_name']}: {row['notes']}")
    print()

In [ ]:
# Compare Hom from different sources
sources = ['subject_catalog', 'author_index', 'methodology_index']

print("Comparing Hom(A, -) for different starting points A:\n")
for source in sources:
    hom = hom_from(source, morphisms)
    targets = hom['morphism_target'].tolist() if len(hom) > 0 else ['(none)']
    print(f"Hom({source}, -) reaches: {targets}")

## Part 2: Representable Functors

A functor F: C → Set is called **representable** if it is naturally isomorphic to some Hom(A, -).

In Dray's words:

> "Special functors Hom(A, -) represent the viewpoint from a single access method. These representables determine all other observations."
> — Lorren Dray, *The Representable Perspective* (Year 938)

In [ ]:
# Find Dray's letter about representables
rep_letter = correspondence[correspondence['key_concepts'].str.contains('representable', case=False, na=False)]

if len(rep_letter) > 0:
    letter = rep_letter.iloc[0]
    print(f"From Dray's Correspondence ({letter['date']}):\n")
    print(f"Subject: {letter['subject']}")
    print(f"\n\"{letter['excerpt']}\"")

## Part 3: The Representable Perspective in Embeddings

In the embedding interpretation:
- Each probe is like a Hom-functor: it asks "how does this document relate to this aspect?"
- A document's response to all probes (all Hom-functors) fully characterizes it

This is a preview of the **Yoneda Lemma**: an object is fully determined by how other objects probe it.

In [ ]:
# Visualize documents through the lens of different probes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Select four probes as "representable perspectives"
probes = ['topic_boundaries', 'topic_categories', 'topic_expeditions', 'topic_creatures']
colors = ['steelblue', 'coral', 'forestgreen', 'purple']

for idx, (probe, color) in enumerate(zip(probes, colors)):
    ax = axes[idx // 2, idx % 2]
    
    probe_data = embeddings[embeddings['probe_name'] == probe].copy()
    
    if len(probe_data) > 0:
        # Sort by value
        probe_data = probe_data.sort_values('numerical_value', ascending=True)
        
        # Get short titles
        titles = [t[:30] + '...' if len(t) > 30 else t for t in probe_data['document_title']]
        values = probe_data['numerical_value'].values
        
        ax.barh(range(len(titles)), values, color=color, alpha=0.7)
        ax.set_yticks(range(len(titles)))
        ax.set_yticklabels(titles, fontsize=8)
        ax.set_xlabel('Functor Value')
        ax.set_xlim(0, 1)
        ax.set_title(f'Representable Perspective: Hom({probe}, -)', fontsize=10)
        ax.grid(True, axis='x', alpha=0.3)

plt.suptitle('Documents Seen Through Different Representable Perspectives', fontsize=12)
plt.tight_layout()
plt.show()

## Part 4: The Yoneda Intuition

The **Yoneda Lemma** (which Strand would later formalize as the "Probing Lemma") states:

> An object A is fully determined by how all other objects probe it — that is, by the functor Hom(-, A).

For documents:
> A document is fully determined by its responses to all probes.

This is why embeddings work: if two documents respond identically to all probes, they are (for practical purposes) the same document.

In [ ]:
# Check if any two documents have identical probe responses
unique_docs = embeddings['document_id'].unique()

# Build profile for each document
profiles = {}
for doc_id in unique_docs:
    doc_emb = embeddings[embeddings['document_id'] == doc_id]
    profile = doc_emb.set_index('probe_name')['numerical_value'].to_dict()
    profiles[doc_id] = profile

# Compare documents
print("Document Profiles (Responses to All Probes):\n")
for doc_id in list(unique_docs)[:3]:
    title = embeddings[embeddings['document_id'] == doc_id]['document_title'].iloc[0]
    print(f"{doc_id}: {title}")
    for probe, value in list(profiles[doc_id].items())[:4]:
        print(f"  {probe}: {value:.2f}")
    print("  ...")
    print()

In [ ]:
# Visualize the Yoneda perspective
fig, ax = plt.subplots(figsize=(12, 6))

# Create a radar chart for two documents
doc1_id = 'DOC-001'
doc2_id = 'DOC-003'

# Get probes and values
doc1_emb = embeddings[embeddings['document_id'] == doc1_id].set_index('probe_name')['numerical_value']
doc2_emb = embeddings[embeddings['document_id'] == doc2_id].set_index('probe_name')['numerical_value']

# Find common probes
common_probes = list(set(doc1_emb.index) & set(doc2_emb.index))

if len(common_probes) >= 3:
    # Set up radar chart
    angles = np.linspace(0, 2 * np.pi, len(common_probes), endpoint=False)
    
    # Get values
    vals1 = [doc1_emb.loc[p] for p in common_probes]
    vals2 = [doc2_emb.loc[p] for p in common_probes]
    
    # Close the loop
    vals1 += [vals1[0]]
    vals2 += [vals2[0]]
    angles_plot = np.concatenate([angles, [angles[0]]])
    
    ax.plot(angles_plot, vals1, 'o-', label=doc1_id, color='steelblue', linewidth=2)
    ax.fill(angles_plot, vals1, alpha=0.2, color='steelblue')
    
    ax.plot(angles_plot, vals2, 's-', label=doc2_id, color='coral', linewidth=2)
    ax.fill(angles_plot, vals2, alpha=0.2, color='coral')
    
    ax.set_xticks(angles)
    ax.set_xticklabels([p.replace('topic_', '') for p in common_probes], fontsize=9)
    ax.set_ylim(0, 1)
    ax.legend(loc='upper right')

ax.set_title('The Yoneda Perspective: Documents Defined by Their Probe Responses', fontsize=12)
plt.tight_layout()
plt.show()

## Part 5: Connection to Vance's Debate

In Year 940, Dray debated with Merrit Vance about how their frameworks related:

> "I see now how our frameworks connect. Your weighted passages are functors into a category enriched over a monoid of weights. My document functors are classical presheaves. Together they form a unified theory."
> — Lorren Dray

In [ ]:
# Find the Dray-Vance debate correspondence
vance_letters = correspondence[correspondence['sender'].isin(['lorren_dray', 'merrit_vance']) & 
                               correspondence['recipient'].isin(['lorren_dray', 'merrit_vance'])]

print("Dray-Vance Correspondence on Unified Theory:\n")
for _, letter in vance_letters.iterrows():
    print(f"Date: {letter['date']}")
    print(f"From: {letter['sender']} → To: {letter['recipient']}")
    print(f"Subject: {letter['subject']}")
    print(f"\n\"{letter['excerpt']}\"")
    print("\n" + "-"*60 + "\n")

## Summary

In this tutorial, we've learned:

1. **Hom-functors** Hom(A, -) give all morphisms from A to other objects
2. **Representable functors** capture "perspectives" from a single vantage point
3. **The Yoneda intuition**: An object is determined by how others probe it
4. **Connection to embeddings**: Probe responses fully characterize documents

### Key Quote

> "Special functors Hom(A, -) represent the viewpoint from a single access method. These representables determine all other observations."
> — Lorren Dray

### Next Tutorial

In Tutorial 7, we'll synthesize Dray's complete framework and trace her legacy through the work of Pelleth Strand and beyond.

---

*Part of the [Category Theory & LLMs Series](https://github.com/buildLittleWorlds)*